In [14]:
# Import necessary libraries
import pandas as pd               # For data manipulation using DataFrames
import numpy as np                # For numerical operations
import matplotlib.pyplot as plt   # For data visualization
import os                         # For operating system-related tasks
import joblib                     # For saving and loading models
import hopsworks                  # For getting access to hopsworks
import re

# Import specific modules from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder   # For data preprocessing
from sklearn.metrics import accuracy_score                        # For evaluating model accuracy

from dotenv import load_dotenv
import os
load_dotenv()

#Connecting to hopsworks
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.


In [15]:
# Load and display the data from CSV to confirm
tsla_df = pd.read_csv('TSLA_stock_price.csv')
print(tsla_df.head())    

         date  1. open   2. high  3. low  4. close   5. volume ticker
0  2024-05-13   170.00  175.4000  169.00    171.89  67018903.0   TSLA
1  2024-05-10   173.05  173.0599  167.75    168.47  72627178.0   TSLA
2  2024-05-09   175.01  175.6200  171.37    171.97  65950292.0   TSLA
3  2024-05-08   171.59  176.0600  170.15    174.72  79969488.0   TSLA
4  2024-05-07   182.40  183.2600  177.40    177.81  75045854.0   TSLA


In [16]:
#Defining a function to clean the column names
def clean_column_name(name):
    # Remove all non-letter characters
    cleaned_name = re.sub(r'[^a-zA-Z]', '', name)
    return cleaned_name

In [17]:
tsla_df

,date,1. open,2. high,3. low,4. close,5. volume,ticker
0,2024-05-13,170.00,175.4000,169.00,171.89,67018903.0,TSLA
1,2024-05-10,173.05,173.0599,167.75,168.47,72627178.0,TSLA
2,2024-05-09,175.01,175.6200,171.37,171.97,65950292.0,TSLA
3,2024-05-08,171.59,176.0600,170.15,174.72,79969488.0,TSLA
4,2024-05-07,182.40,183.2600,177.40,177.81,75045854.0,TSLA
...,...,...,...,...,...,...,...
3487,2010-07-06,20.00,20.0000,15.83,16.11,6866900.0,TSLA
3488,2010-07-02,23.00,23.1000,18.71,19.20,5139800.0,TSLA
3489,2010-07-01,25.00,25.9200,20.27,21.96,8218800.0,TSLA
3490,2010-06-30,25.79,30.4192,23.30,23.83,17187100.0,TSLA


In [18]:
# Cleaning up column names for 'tsla_df'
tsla_df.columns = [clean_column_name(col) for col in tsla_df.columns]
print(tsla_df.columns)

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'ticker'], dtype='object')


In [19]:
# Converting the "date" column to timestamp
tsla_df['date'] = pd.to_datetime(tsla_df['date'])

In [20]:
# Defining the stocks feature group
tesla_fg = fs.get_or_create_feature_group(
    name="tesla_stock",
    description="Tesla stock dataset from alpha vantage",
    version=5,
    primary_key=["ticker"],
    event_time=['date'],
    online_enabled=False,
)

2024-05-14 12:11:21,958 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [21]:
#Inserting the stock data into the stocks feature group
tesla_fg.insert(tsla_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/693399/fs/689222/fg/813389


Uploading Dataframe: 0.00% |          | Rows 0/3492 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: tesla_stock_5_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/tesla_stock_5_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x194d7d8bc50>, None)

In [22]:
#Collecting news df
news_df = pd.read_csv('news_articles_ema.csv')

In [23]:
#Dropping exp mean 7 days
news_df_updated = news_df.drop(columns=['exp_mean_7_days'])

In [24]:
#Updating date to datetime
news_df_updated['date'] = pd.to_datetime(news_df_updated['date'])

In [25]:
#Defining the news feature group
news_sentiment_fg = fs.get_or_create_feature_group(
    name='news_sentiment_updated',
    description='News sentiment from Polygon',
    version=5,
    primary_key=['ticker'],
    event_time=['date'],
    online_enabled=False,
)

2024-05-14 12:11:37,246 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [26]:
#Inserting the news data into the news feature group
news_sentiment_fg.insert(news_df_updated)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/693399/fs/689222/fg/814412


Uploading Dataframe: 0.00% |          | Rows 0/74 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: news_sentiment_updated_5_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/news_sentiment_updated_5_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x194d7dab290>, None)